Clone the AMIE repository available here: 
https://github.com/lajus/amie

Dataset we are gonna use in the lab: 
https://github.com/wencolani/IterE/blob/master/datasets/FB15k-237-sparse/train.txt

To run AMIE, we need java. If you use colab, you need the following.
Prepare java kernel for google colab
Following https://colab.research.google.com/github/aws-samples/d2l-java/blob/colab/chapter_installation/index.ipynb#scrollTo=YHfNWsXhj14C
Run the command below (You may need to restart the notebook afterwards)
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

In [4]:
# Use ! prefix to run shell commands
!java -version

openjdk version "15" 2020-09-15
OpenJDK Runtime Environment AdoptOpenJDK (build 15+36)
OpenJDK 64-Bit Server VM AdoptOpenJDK (build 15+36, mixed mode, sharing)


In [5]:
!ls

DAY3.ipynb    amie-output   amie-rules    fb15k-237.txt rules


In [6]:
!head fb15k-237.txt

In [7]:
# Run amie following instructions here https://github.com/lajus/amie
!java -XX:-UseGCOverheadLimit -Xmx16G -jar /Users/ilaria/amie/amie-dev.jar fb15k-237.txt 2>amie-output >amie-rules

In [8]:
!wc -l amie-output

       1 amie-output


In [9]:
cat amie-output

Using the default schema relations


In [10]:
!wc -l amie-rules

    7609 amie-rules


In [11]:
!head -n30 amie-rules

Assuming rdf:type as type relation
Loading files... 
  Starting fb15k-237.txt
  Finished fb15k-237.txt, still running: 0
Loaded 272115 facts in 1.61 s using 567 MB
Using HeadCoverage as pruning metric with minimum threshold 0.01
Using recursivity limit 3
Lazy mining assistant that stops counting when the denominator gets too high
No minimum threshold on standard confidence
Filtering on PCA confidence with minimum threshold 0.1
Constants in the arguments of relations are disabled
Lossless (query refinement) heuristics enabled
MRT calls: 0
Starting the mining phase... Using 8 threads
Rule	Head Coverage	Std Confidence	PCA Confidence	Positive Examples	Body size	PCA Body size	Functional variable
?b  /base/popstra/celebrity/friendship./base/popstra/friendship/participant  ?a   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.036144578	0.00397088	0.206896552	6	1511	29	?b
?b  /people/person/sibling_s./people/sibling_relationship/sibling  ?a   => ?a  /influence/in

In [12]:
!grep "=>" amie-rules >rules

In [13]:
!head rules

?b  /base/popstra/celebrity/friendship./base/popstra/friendship/participant  ?a   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.036144578	0.00397088	0.206896552	6	1511	29	?b
?b  /people/person/sibling_s./people/sibling_relationship/sibling  ?a   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.012048193	0.015267176	0.5	2	131	4	?b
?a  /base/popstra/celebrity/friendship./base/popstra/friendship/participant  ?b   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.036144578	0.00397088	0.222222222	6	1511	27	?b
?b  /influence/influence_node/peers./influence/peer_relationship/peers  ?a   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.78313253	0.78313253	0.860927152	130	166	151	?b
?a  /people/person/sibling_s./people/sibling_relationship/sibling  ?b   => ?a  /influence/influence_node/peers./influence/peer_relationship/peers  ?b	0.012048193	0.015267176	0.4	2	131	5	?b
?b

In [14]:
# Load KG by relations 
import pandas as pd
from collections import defaultdict
kgfile = "fb15k-237.txt"
rulefile = "rules"
kg = pd.read_csv(kgfile,sep='\t',header=None)

In [15]:
len(kg)

272115

In [17]:
kg[0:5]

,0,1,2
0,/m/027rn,/location/country/form_of_government,/m/06cx9
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8


In [18]:
# load kg a relation 2 valid triples dictionary
relation2triples = defaultdict(list)
for i in kg.iterrows():
    source = i[1][0]
    tail = i[1][2]
    relation = i[1][1]
    relation2triples[relation].append((source, tail))
len(relation2triples)

237

In [19]:
# load each rule as a tuple of 2 element: body (a list of triples) and head (a single triple)
# body will contain either 1 or 2 triples
def group_elements(n, iterable, padvalue='?'):
    from itertools import zip_longest
    return list(zip_longest(*[iter(iterable)]*n, fillvalue=padvalue))

def load_rules(rulefile):
    with open(rulefile) as f:
        rlines = f.readlines()
        print(len(rlines))
        rules = []
        rleng = []
        for k,rline in enumerate(rlines):
            rule = rline.split('\t')[0]
            body = rule.split('=>')[0]
            head = rule.split('=>')[1]
            body_elements = body.split(' ')
            body_elements = list(filter(lambda x: x != "", body_elements))
            head_elements = head.split(' ')
            head_elements = list(filter(lambda x: x != "", head_elements))
            
            body_triples = group_elements(3,body_elements)
            head_triple = group_elements(3,head_elements)[0]
            rleng.append(len(body_triples))         
            rules.append((body_triples, head_triple))
    return (rules, rleng)

In [20]:
(rules, rleng) = load_rules(rulefile)

7591


In [21]:
max(rleng)

2

In [22]:
rules[0:5]

[([('?b',
    '/base/popstra/celebrity/friendship./base/popstra/friendship/participant',
    '?a')],
  ('?a',
   '/influence/influence_node/peers./influence/peer_relationship/peers',
   '?b')),
 ([('?b',
    '/people/person/sibling_s./people/sibling_relationship/sibling',
    '?a')],
  ('?a',
   '/influence/influence_node/peers./influence/peer_relationship/peers',
   '?b')),
 ([('?a',
    '/base/popstra/celebrity/friendship./base/popstra/friendship/participant',
    '?b')],
  ('?a',
   '/influence/influence_node/peers./influence/peer_relationship/peers',
   '?b')),
 ([('?b',
    '/influence/influence_node/peers./influence/peer_relationship/peers',
    '?a')],
  ('?a',
   '/influence/influence_node/peers./influence/peer_relationship/peers',
   '?b')),
 ([('?a',
    '/people/person/sibling_s./people/sibling_relationship/sibling',
    '?b')],
  ('?a',
   '/influence/influence_node/peers./influence/peer_relationship/peers',
   '?b'))]

In [23]:

def instantiate_rule(relation2triples, rule):
    rule_body_triples = rule[0]
    rule_head_triple = rule[1]
    
    instantiated_rules = []
    new_inferred_triples = []
    
    if len(rule_body_triples) == 1:
        body_source = rule_body_triples[0][0]
        body_relation = rule_body_triples[0][1]
        body_tail = rule_body_triples[0][2]
        
        head_source = rule_head_triple[0]
        head_relation = rule_head_triple[1]
        head_tail = rule_head_triple[2]
        
        body_relation_triples = relation2triples[body_relation]

        
        for (source, tail) in body_relation_triples:
            if (body_source == head_source):
                rule_instance = '%s %s %s => %s %s %s' % (source, body_relation, tail, source, head_relation, tail)
                instantiated_rules.append(rule_instance)
              
                if (source, tail) not in relation2triples[head_relation]:
                    new_inferred_triples.append((source, head_relation, tail))
            else:
                rule_instance = '%s %s %s => %s %s %s' % (source, body_relation, tail, tail, head_relation, source)
                instantiated_rules.append(rule_instance)
                if (tail, source) not in relation2triples[head_relation]:
                    new_inferred_triples.append((tail, head_relation, source))
                
    else: # body has 2 triples
        body_first_source = rule_body_triples[0][0]
        body_first_relation = rule_body_triples[0][1]
        body_first_tail = rule_body_triples[0][2]
        
        body_second_source = rule_body_triples[1][0]
        body_second_relation = rule_body_triples[1][1]
        body_second_tail = rule_body_triples[1][2]
        
        body_first_triples = relation2triples[body_first_relation]
        
        body_second_triples = relation2triples[body_second_relation]
        
        head_source = rule_head_triple[0]
        head_relation = rule_head_triple[1]
        head_tail = rule_head_triple[2]
        
        distinct_variables = list(set([body_first_source, body_first_tail, body_second_source, body_second_tail]))
        print(len(distinct_variables))
        
        derived_pairs = []
        
        
        for (first_source, first_tail) in body_first_triples:
            for (second_source, second_tail) in body_second_triples:
                valid = 1
               
                if body_first_source == body_second_source and first_source != second_source:
                    valid = 0
                                                    
                if body_first_source == body_second_tail and first_source != second_tail:
                    valid = 0
                                                
                if body_first_tail == body_second_source and first_tail != second_source:
                    valid = 0
            
                if body_first_tail == body_second_tail and first_tail != second_tail:
                    valid = 0
                        
                if valid == 0:
                    continue
                        
                
                hsource = ''
                htail = ''
                
                if head_source == body_first_source:
                    hsource = first_source
                elif head_source == body_first_tail:
                    hsource = first_tail
                elif head_source == body_second_source:
                    hsource = second_source
                elif head_source == body_second_tail:
                    hsource = second_tail
                        
                if head_tail == body_first_source:
                    htail = first_source
                elif head_tail == body_first_tail:
                    htail = first_tail
                elif head_tail == body_second_source:
                    htail = second_source
                elif head_tail == body_second_source:
                    htail = second_tail
                
         
                
                rule_instance = '%s %s %s AND %s %s %s => %s %s %s' %  (first_source, body_first_relation, first_tail, second_source, body_second_relation, second_tail, hsource, head_relation, htail)
                instantiated_rules.append(rule_instance)
      
                if (hsource, htail) not in relation2triples[head_relation]:
                    new_inferred_triples.append((hsource, head_relation, htail))

    
    return (instantiated_rules, new_inferred_triples)
                
        
        
        
    

In [25]:
all_instances = []
all_new_triples = []
import time
start_time = time.time()
for k, rule in enumerate(rules):
    print('Processing RULE # %d \n'% k)
    print(rule)
    (instances, new_triples) = instantiate_rule(relation2triples, rule)
    print('Found Instances: %d\n' % (len(instances)))
    print('New triples inferred: %d\n' % (len(new_triples)))
    all_instances.extend(instances)
    all_new_triples.extend(new_triples)
elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
print(f"Took: {elapsed}")


  

Processing RULE # 0 

([('?b', '/base/popstra/celebrity/friendship./base/popstra/friendship/participant', '?a')], ('?a', '/influence/influence_node/peers./influence/peer_relationship/peers', '?b'))
Found Instances: 1511

New triples inferred: 1505

Processing RULE # 1 

([('?b', '/people/person/sibling_s./people/sibling_relationship/sibling', '?a')], ('?a', '/influence/influence_node/peers./influence/peer_relationship/peers', '?b'))
Found Instances: 131

New triples inferred: 129

Processing RULE # 2 

([('?a', '/base/popstra/celebrity/friendship./base/popstra/friendship/participant', '?b')], ('?a', '/influence/influence_node/peers./influence/peer_relationship/peers', '?b'))
Found Instances: 1511

New triples inferred: 1505

Processing RULE # 3 

([('?b', '/influence/influence_node/peers./influence/peer_relationship/peers', '?a')], ('?a', '/influence/influence_node/peers./influence/peer_relationship/peers', '?b'))
Found Instances: 166

New triples inferred: 36

Processing RULE # 4 

([

Found Instances: 4197

New triples inferred: 4147

Processing RULE # 42 

([('?a', '/people/person/employment_history./business/employment_tenure/company', '?b')], ('?a', '/government/politician/government_positions_held./government/government_position_held/jurisdiction_of_office', '?b'))
Found Instances: 316

New triples inferred: 303

Processing RULE # 43 

([('?b', '/base/localfood/seasonal_month/produce_available./base/localfood/produce_availability/seasonal_months', '?a')], ('?a', '/base/localfood/seasonal_month/produce_available./base/localfood/produce_availability/seasonal_months', '?b'))
Found Instances: 108

New triples inferred: 18

Processing RULE # 44 

([('?b', '/award/award_nominee/award_nominations./award/award_nomination/nominated_for', '?a')], ('?a', '/film/film/film_production_design_by', '?b'))
Found Instances: 6277

New triples inferred: 6214

Processing RULE # 45 

([('?a', '/film/film/costume_design_by', '?b')], ('?a', '/film/film/film_production_design_by', '?b')


New triples inferred: 112

Processing RULE # 81 

([('?b', '/base/popstra/celebrity/dated./base/popstra/dated/participant', '?a')], ('?a', '/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant', '?b'))
Found Instances: 1404

New triples inferred: 1293

Processing RULE # 82 

([('?b', '/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant', '?a')], ('?a', '/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant', '?b'))
Found Instances: 449

New triples inferred: 81

Processing RULE # 83 

([('?b', '/base/popstra/celebrity/breakup./base/popstra/breakup/participant', '?a')], ('?a', '/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant', '?b'))
Found Instances: 153

New triples inferred: 129

Processing RULE # 84 

([('?b', '/celebrities/celebrity/sexual_relationships./celebrities/romantic_relationship/celebrity', '?a')], ('?a', '/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant', '?b'))
Found Instances: 

Found Instances: 859

New triples inferred: 735

Processing RULE # 116 

([('?b', '/film/film/written_by', '?a')], ('?a', '/film/actor/film./film/performance/film', '?b'))
Found Instances: 787

New triples inferred: 640

Processing RULE # 117 

([('?b', '/film/film/produced_by', '?a')], ('?a', '/film/actor/film./film/performance/film', '?b'))
Found Instances: 1285

New triples inferred: 1138

Processing RULE # 118 

([('?b', '/award/award_winning_work/awards_won./award/award_honor/award_winner', '?a')], ('?a', '/film/actor/film./film/performance/film', '?b'))
Found Instances: 2992

New triples inferred: 2029

Processing RULE # 119 

([('?a', '/award/award_nominee/award_nominations./award/award_nomination/nominated_for', '?b')], ('?a', '/film/actor/film./film/performance/film', '?b'))
Found Instances: 6277

New triples inferred: 4171

Processing RULE # 120 

([('?a', '/film/film/country', '?b')], ('?a', '/film/film/distributors./film/film_film_distributor_relationship/region', '?b'))
Fo

Found Instances: 5201

New triples inferred: 4986

Processing RULE # 161 

([('?a', '/base/biblioness/bibs_location/state', '?b')], ('?a', '/location/capital_of_administrative_division/capital_of./location/administrative_division_capital_relationship/administrative_division', '?b'))
Found Instances: 300

New triples inferred: 261

Processing RULE # 162 

([('?b', '/olympics/olympic_sport/athletes./olympics/olympic_athlete_affiliation/olympics', '?a')], ('?a', '/user/jg/default_domain/olympic_games/sports', '?b'))
Found Instances: 381

New triples inferred: 163

Processing RULE # 163 

([('?a', '/olympics/olympic_games/sports', '?b')], ('?a', '/user/jg/default_domain/olympic_games/sports', '?b'))
Found Instances: 664

New triples inferred: 181

Processing RULE # 164 

([('?b', '/olympics/olympic_games/sports', '?a')], ('?a', '/olympics/olympic_sport/athletes./olympics/olympic_athlete_affiliation/olympics', '?b'))
Found Instances: 664

New triples inferred: 402

Processing RULE # 165 

(

...

Found Instances: 560

New triples inferred: 421

Processing RULE # 7444 

([('?g', '/award/award_winning_work/awards_won./award/award_honor/award_winner', '?b'), ('?g', '/film/film/produced_by', '?a')], ('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b'))
3
Found Instances: 1931

New triples inferred: 1714

Processing RULE # 7445 

([('?g', '/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor', '?a'), ('?g', '/tv/tv_program/regular_cast./tv/regular_tv_appearance/actor', '?b')], ('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b'))
3
Found Instances: 10506

New triples inferred: 10506

Processing RULE # 7446 

([('?a', '/award/award_nominee/award_nominations./award/award_nomination/award_nominee', '?h'), ('?b', '/award/award_nominee/award_nominations./award/award_nomination/award_nominee', '?h')], ('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b'))
3
Found Instances: 209555

New triples inferred: 16

Found Instances: 206400

New triples inferred: 157880

Processing RULE # 7472 

([('?g', '/award/award_nominee/award_nominations./award/award_nomination/award_nominee', '?a'), ('?g', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b')], ('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b'))
3
Found Instances: 103746

New triples inferred: 103746

Processing RULE # 7473 

([('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?h'), ('?h', '/people/deceased_person/place_of_death', '?b')], ('?a', '/people/deceased_person/place_of_death', '?b'))
3
Found Instances: 370

New triples inferred: 370

Processing RULE # 7474 

([('?g', '/award/award_winner/awards_won./award/award_honor/award_winner', '?a'), ('?g', '/people/deceased_person/place_of_death', '?b')], ('?a', '/people/deceased_person/place_of_death', '?b'))
3
Found Instances: 380

New triples inferred: 380

Processing RULE # 7475 

([('?g', '/award/award_nominee/award_no

Found Instances: 1934

New triples inferred: 1554

Processing RULE # 7501 

([('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?h'), ('?h', '/common/topic/webpage./common/webpage/category', '?b')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 1714

New triples inferred: 1714

Processing RULE # 7502 

([('?a', '/award/award_nominee/award_nominations./award/award_nomination/nominated_for', '?h'), ('?h', '/common/topic/webpage./common/webpage/category', '?b')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 1443

New triples inferred: 1443

Processing RULE # 7503 

([('?a', '/base/schemastaging/person_extra/net_worth./measurement_unit/dated_money_value/currency', '?h'), ('?h', '/common/topic/webpage./common/webpage/category', '?b')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 475

New triples inferred: 475

Processing RULE # 7504 

([('?a', '/base/pops

Found Instances: 285

New triples inferred: 285

Processing RULE # 7530 

([('?g', '/common/topic/webpage./common/webpage/category', '?b'), ('?g', '/music/record_label/artist', '?a')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 1462

New triples inferred: 413

Processing RULE # 7531 

([('?g', '/award/award_nominee/award_nominations./award/award_nomination/award_nominee', '?a'), ('?g', '/award/award_nominee/award_nominations./award/award_nomination/award_nominee', '?b')], ('?a', '/award/award_winner/awards_won./award/award_honor/award_winner', '?b'))
3
Found Instances: 209141

New triples inferred: 209141

Processing RULE # 7532 

([('?h', '/common/topic/webpage./common/webpage/category', '?b'), ('?a', '/location/location/contains', '?h')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 2611

New triples inferred: 1967

Processing RULE # 7533 

([('?g', '/common/topic/webpage./common/webpage/category', '?b'), (

Found Instances: 581

New triples inferred: 467

Processing RULE # 7560 

([('?b', '/dataworld/gardening_hint/split_to', '?h'), ('?a', '/music/artist/track_contributions./music/track_contribution/role', '?h')], ('?a', '/people/person/profession', '?b'))
3
Found Instances: 602

New triples inferred: 471

Processing RULE # 7561 

([('?b', '/dataworld/gardening_hint/split_to', '?h'), ('?h', '/music/instrument/instrumentalists', '?a')], ('?a', '/people/person/profession', '?b'))
3
Found Instances: 1267

New triples inferred: 1043

Processing RULE # 7562 

([('?g', '/common/topic/webpage./common/webpage/category', '?b'), ('?g', '/location/location/adjoin_s./location/adjoining_relationship/adjoins', '?a')], ('?a', '/common/topic/webpage./common/webpage/category', '?b'))
3
Found Instances: 276

New triples inferred: 118

Processing RULE # 7563 

([('?a', '/base/popstra/celebrity/friendship./base/popstra/friendship/participant', '?h'), ('?h', '/people/person/profession', '?b')], ('?a', '/peopl

In [26]:
len(all_new_triples)

16146905

In [28]:
unique_triples = list(set(all_new_triples))
len(unique_triples)

2622354

In [29]:
len(kg)

272115

In [30]:
for triple in all_new_triples[0:100]:
    print(triple)

('/m/04t2l2', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/01xdf5')
('/m/0prfz', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/0bl2g')
('/m/03m8lq', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/05gml8')
('/m/0mdqp', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/018db8')
('/m/02lnhv', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/01vrt_c')
('/m/01j4ls', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/01k8rb')
('/m/01vhb0', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/011zd3')
('/m/0jfx1', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/0237fw')
('/m/0bsb4j', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/014zcr')
('/m/01w02sy', '/influence/influence_node/peers./influence/peer_relationship/peers', '/m/0693l')
('/m/07g2v', '/influence/influenc